# Data Preparation

**Author:** Florian Klaver

In this Notebook the datasets are prepared fo the Multi-Criteria Analysis (MCA).

The following steps are performed: 
1. Calculate Slope from the DEM.
2. Rasterize Forest areas (Cover).
3. Rasterize Water bodies (Mask).
4. Filter and rasterize heavy traffic Roads and calculates Euclidean Distance.
5. Filter and rasterize Settlement areas and calculates Euclidean Distance.
6. Extract and rasterize Prey potential (Alpine pastures).

All outputs are saved as 10m aligned GeoTIFFs in the 'output' directory.

## Setup

In [1]:
import os
import geopandas as gpd
import rasterio
from rasterio import features
from scipy.ndimage import distance_transform_edt
from osgeo import gdal
import numpy as np
from pathlib import Path

In [2]:
# --- PATH CONFIGURATION ---
try:
    # Try to get the script directory (works for standard .py files)
    script_dir = Path(__file__).parent
except NameError:
    # Fallback for Jupyter Notebooks
    script_dir = Path.cwd()

PROJECT_ROOT = script_dir.parent
DATA_DIR = PROJECT_ROOT / 'data'
OUTPUT_DIR = PROJECT_ROOT / 'output'

# Input Files
DEM_PATH = OUTPUT_DIR / "dem_10m_graubuenden.tif"
BOUNDARIES_PATH = DATA_DIR / "swissBOUNDARIES3D_1_5_LV95_LN02.gpkg"
TLM_PATH = DATA_DIR / "SWISSTLM3D_2025.gpkg"
AREAL_PATH = DATA_DIR / "arealstatistik_2056.gpkg"

# Output Files
SLOPE_PATH = OUTPUT_DIR / "slope_10m_graubuenden.tif"
FOREST_RASTER_PATH = OUTPUT_DIR / "forest_10m_graubuenden.tif"
ROADS_DIST_PATH = OUTPUT_DIR / "distance_roads_10m.tif"
SETTLEMENT_DIST_PATH = OUTPUT_DIR / "distance_settlements_10m.tif"
WATER_MASK_PATH = OUTPUT_DIR / "water_mask_10m.tif"
PREY_RASTER_PATH = OUTPUT_DIR / "prey_10m.tif"

# Ensure output directory exists
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

## Process

In [3]:
# --- HELPER FUNCTIONS ---

def safe_delete(path):
    """
    Safely deletes a file if it exists. 
    Raises a clear error if the file is locked by another process (e.g., QGIS).
    """
    path = Path(path)
    if path.exists():
        try:
            os.remove(path)
        except PermissionError:
            print(f"CRITICAL ERROR: Could not delete {path.name}. File is locked.")
            print("Action required: Close QGIS or any other software using this file.")
            raise PermissionError(f"File locked: {path}")
        except Exception as e:
            print(f"Error deleting {path.name}: {e}")

def get_graubuenden_boundary():
    """
    Loads and returns the dissolved geometry of Canton Graubünden.
    Used to spatially mask vector data loading to improve performance.
    """
    print("-> Loading Cantonal Boundary (Graubünden)...")
    try:
        gdf = gpd.read_file(BOUNDARIES_PATH, layer='TLM_KANTONSGEBIET')
        
        # Filter logic to handle different attribute names in datasets
        if 'NAME' in gdf.columns:
            gdf_gr = gdf[gdf['NAME'].isin(['Graubünden', 'Grigioni', 'Grischun'])]
        elif 'name' in gdf.columns:
            gdf_gr = gdf[gdf['name'].isin(['Graubünden', 'Grigioni', 'Grischun'])]
        elif 'kantonsnummer' in gdf.columns:
            gdf_gr = gdf[gdf['kantonsnummer'] == 18]
        else:
            print("Warning: Could not filter for Graubünden. Using full dataset.")
            gdf_gr = gdf

        # Dissolve geometry
        try:
            return gdf_gr.geometry.union_all()
        except AttributeError:
            return gdf_gr.geometry.unary_union # Fallback for older geopandas versions
            
    except Exception as e:
        print(f"Error loading boundaries: {e}")
        return None

def simple_rasterize(gdf, reference_path, output_path):
    """
    Rasterizes a GeoDataFrame to a binary raster (1=Feature, 0=Background).
    Uses the reference_path (DEM) for alignment and resolution.
    """
    output_path = Path(output_path)
    safe_delete(output_path) 
    
    with rasterio.open(reference_path) as src:
        meta = src.meta.copy()
        meta.update(dtype=rasterio.uint8, count=1, nodata=0)
        shape = src.shape
        transform = src.transform
        
    print(f"   ... Rasterizing {len(gdf)} features (Binary)...")
    
    if len(gdf) > 0:
        shapes = ((geom, 1) for geom in gdf.geometry)
        arr = features.rasterize(
            shapes=shapes, 
            out_shape=shape, 
            transform=transform, 
            fill=0, 
            default_value=1, 
            dtype=rasterio.uint8
        )
    else:
        print("   Warning: No features found. Creating empty raster.")
        arr = np.zeros(shape, dtype=rasterio.uint8)
    
    with rasterio.open(output_path, 'w', **meta) as dst:
        dst.write(arr, 1)
    print(f"   Success: {output_path.name} saved.")

def rasterize_and_distance(gdf, reference_path, output_path, burn_value=1):
    """
    Rasterizes features and calculates the Euclidean Distance to them.
    Output is a float32 raster representing distance in meters.
    """
    output_path = Path(output_path)
    safe_delete(output_path)
    
    with rasterio.open(reference_path) as src:
        meta = src.meta.copy()
        shape = src.shape
        transform = src.transform
        cell_size = src.res[0]

    print(f"   ... Rasterizing {len(gdf)} features...")
    if len(gdf) > 0:
        shapes = ((geom, burn_value) for geom in gdf.geometry)
        binary = features.rasterize(
            shapes=shapes, 
            out_shape=shape, 
            transform=transform, 
            default_value=burn_value, 
            dtype=rasterio.uint8
        )
    else:
        binary = np.zeros(shape, dtype=rasterio.uint8)

    print("   ... Calculating Euclidean Distance...")
    # Invert binary mask: 0 (background) becomes 1 for distance calculation
    # distance_transform_edt calculates distance to the nearest non-zero pixel
    mask = np.logical_not(binary) 
    dist_meters = distance_transform_edt(mask) * cell_size

    meta.update(dtype=rasterio.float32, count=1, nodata=-9999)
    with rasterio.open(output_path, 'w', **meta) as dst:
        dst.write(dist_meters.astype(rasterio.float32), 1)
    print(f"   Success: {output_path.name} saved.")

In [4]:
# ==========================================
# MAIN EXECUTION
# ==========================================

# Check required inputs
if not DEM_PATH.exists():
    print(f"Critical Error: DEM not found at {DEM_PATH}. Run '02_preprocess_dem.py' first.")
    exit()

# Load Mask (Graubünden Geometry)
gr_geometry = get_graubuenden_boundary()
if gr_geometry is None: 
    exit()

# --- 1. CALCULATE SLOPE ---
print("\n--- 1. SLOPE CALCULATION ---")
if not SLOPE_PATH.exists():
    # Use GDAL DEMProcessing for efficient slope calculation
    gdal.DEMProcessing(
        str(SLOPE_PATH), 
        str(DEM_PATH), 
        "slope", 
        options=gdal.DEMProcessingOptions(computeEdges=True, slopeFormat="degree")
    )
    print("Slope calculated.")
else: 
    print("Slope file already exists. Skipping.")

# --- 2. FOREST (COVER) ---
print("\n--- 2. FOREST RASTERIZATION ---")
# Load 'tlm_bb_bodenbedeckung' masked to Graubünden
gdf_land = gpd.read_file(TLM_PATH, layer='tlm_bb_bodenbedeckung', mask=gr_geometry)

# Filter for forest types
forest_types = ['Wald', 'Gebueschwald', 'Wald offen', 'Gehoelzflaeche']
gdf_forest = gdf_land[gdf_land['objektart'].isin(forest_types)]
simple_rasterize(gdf_forest, DEM_PATH, FOREST_RASTER_PATH)

# --- 3. WATER (MASK) ---
print("\n--- 3. WATER RASTERIZATION ---")
# Reuse loaded land cover layer
gdf_water = gdf_land[gdf_land['objektart'] == 'Stehende Gewaesser']
simple_rasterize(gdf_water, DEM_PATH, WATER_MASK_PATH)

# --- 4. ROADS (DISTURBANCE) ---
print("\n--- 4. ROAD DISTANCE CALCULATION ---")
gdf_roads = gpd.read_file(TLM_PATH, layer='tlm_strassen_strasse', mask=gr_geometry)

# Filter for heavy traffic roads
heavy_types = [
    'Autobahn', 'Autostrasse', '10m Strasse', '8m Strasse', '6m Strasse', 
    'Hauptstrasse', 'Verbindungsstrasse', 'Einfahrt', 'Ausfahrt', 'Zufahrt'
]
# Exclude tunnels (they are underground and do not disturb surface wildlife)
gdf_surf = gdf_roads[
    gdf_roads['objektart'].isin(heavy_types) & 
    ~gdf_roads['kunstbaute'].isin(['Tunnel', 'Unterfuehrung'])
]

if not gdf_surf.empty:
    rasterize_and_distance(gdf_surf, DEM_PATH, ROADS_DIST_PATH)
else:
    print("Warning: No relevant road features found.")

# --- 5. SETTLEMENTS (DISTURBANCE) ---
print("\n--- 5. SETTLEMENT DISTANCE CALCULATION ---")
AS_COLUMN = 'AS18_72' # Using Arealstatistik 2018 (72 Categories)
gdf_areal = gpd.read_file(AREAL_PATH, layer='arealstatistik_all', mask=gr_geometry)

if AS_COLUMN in gdf_areal.columns:
    # Codes 1-13 represent settlements and buildings
    settlement_codes = list(range(1, 14))
    gdf_settle = gdf_areal[gdf_areal[AS_COLUMN].isin(settlement_codes)]
    
    if not gdf_settle.empty:
        print(f"   ... Buffering {len(gdf_settle)} settlement points (150m)...")
        # Buffer points to create a continuous settlement area
        gdf_settle_poly = gpd.GeoDataFrame(geometry=gdf_settle.buffer(150)) 
        rasterize_and_distance(gdf_settle_poly, DEM_PATH, SETTLEMENT_DIST_PATH)
    else:
        print("Warning: No settlement points found.")
else:
    print(f"Error: Column {AS_COLUMN} not found in Arealstatistik.")

# --- 6. PREY (ALPINE PASTURES) ---
print("\n--- 6. PREY POTENTIAL RASTERIZATION ---")
if AS_COLUMN in gdf_areal.columns:
    # Codes 45-49: Alpine pastures and meadows (Prey habitat)
    target_codes = [45, 46, 47, 48, 49]
    print(f"   Filtering codes: {target_codes}")
    
    gdf_prey = gdf_areal[gdf_areal[AS_COLUMN].isin(target_codes)]
    print(f"   -> Found {len(gdf_prey)} prey location points.")
    
    if not gdf_prey.empty:
        # Simple rasterization (Points). Smoothing will be done in the analysis phase.
        simple_rasterize(gdf_prey, DEM_PATH, PREY_RASTER_PATH)
    else:
        print("Warning: No prey data found.")

print("\n--- PREPROCESSING COMPLETED ---")

-> Loading Cantonal Boundary (Graubünden)...

--- 1. SLOPE CALCULATION ---
Slope file already exists. Skipping.

--- 2. FOREST RASTERIZATION ---
   ... Rasterizing 58825 features (Binary)...
   Success: forest_10m_graubuenden.tif saved.

--- 3. WATER RASTERIZATION ---
   ... Rasterizing 3593 features (Binary)...
   Success: water_mask_10m.tif saved.

--- 4. ROAD DISTANCE CALCULATION ---
   ... Rasterizing 5647 features...
   ... Calculating Euclidean Distance...
   Success: distance_roads_10m.tif saved.

--- 5. SETTLEMENT DISTANCE CALCULATION ---
   ... Buffering 6641 settlement points (150m)...
   ... Rasterizing 6641 features...
   ... Calculating Euclidean Distance...
   Success: distance_settlements_10m.tif saved.

--- 6. PREY POTENTIAL RASTERIZATION ---
   Filtering codes: [45, 46, 47, 48, 49]
   -> Found 160112 prey location points.
   ... Rasterizing 160112 features (Binary)...
   Success: prey_10m.tif saved.

--- PREPROCESSING COMPLETED ---
